# Cat and dog avec augmentation

## Initialisations

### Divers

In [1]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import model_selection

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn import datasets

### Keras

In [3]:
import tensorflow as tf

from tensorflow.keras.models import Sequential, load_model

from tensorflow.keras.layers import InputLayer, Dense, Dropout, Flatten

from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPool2D

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator

In [4]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomFlip, RandomRotation, RandomZoom, RandomContrast, RandomTranslation

### Fonctions utiles

In [5]:
def plot_scores(train) :
    accuracy = train.history['accuracy']
    val_accuracy = train.history['val_accuracy']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Score apprentissage')
    plt.plot(epochs, val_accuracy, 'r', label='Score validation')
    plt.title('Scores')
    plt.legend()
    plt.show()

## Transformations d'images

In [ ]:
img = load_img('../input/cat-and-dog/training_set/training_set/cats/cat.1.jpg')

In [ ]:
plt.imshow(img)

Les *couches d'augmentation* (*augmentation layers*) de Keras sont des couches particulières d'un modèle Keras qui permettents de transformer les images d'un *batch* d'images (ou d'un dataset) 
- **RandomFlip(*mode*)** : retourne une image ; *mode* peut être "horizontal", "vertical", ou "horizontal_and_vertical"
- **RandomRotation(*factor*)** : effectue une rotation de l'image de *factor**2*Pi
- **RandomZoom(*height_factor*)** : effectue un zoom ; par exemple RandomZoom((-0.1,0.1)) effectue un zoom aléatoire entre -10% et +10%
- **RandomContrast(*factor*)** : ajuste le contraste
- **RandomTranslation(*height_factor*,*width_factor*)** : décale l'image de *width_factor* horizontalement et *height_factor* verticalement 
 
 https://keras.io/guides/preprocessing_layers/

In [ ]:
data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(1./16),
    RandomZoom((-0.1,0.1)),c  
    RandomContrast(0.2),  
    RandomTranslation(0.1,0.1)
])  

On transforme l'image en un *batch* d'une image en ajoutant une dimension :

In [ ]:
batch = np.expand_dims(img,0)

On peut générer autant d'images transformées qu'on le souhaite :

In [ ]:
for i in range(9):
  augmented_image = data_augmentation(batch)
  plt.imshow(augmented_image[0])
  plt.axis("off")
  plt.show()

## Lecture des images depuis un répertoire

**image_dataset_from_directory** crée un *dataset Tensorflow* à partir des images d'un répertoire donné.  

On suppose que les images sont rangées dans des sous-répertoires, dont les noms donnent les noms de classes (attribut **class_names**)  

Les images sont lues en mémoire par *batch* (32 iamges par défaut)

In [ ]:
train_data_dir = "../input/cat-and-dog/training_set/training_set"
image_size = (299, 299)

dataset = image_dataset_from_directory(
    train_data_dir,
    image_size=image_size,
)

**dataset.take(1)** génère un premier batch (32 images par défaut) :

In [ ]:
plt.figure(figsize=(15, 25))
class_names = dataset.class_names
for images, labels in dataset.take(1):
    for i in range(32):
        plt.subplot(7, 5, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

## Création des datasets 

On peut créer des datasets d'apprentissage et de validation à partir du même répertoire avec l'option **validation_split=0.2** (20% des données pour la validation)  
Dans ce cas, il faut donner la même valeur **seed** (initialisation du générateur aléatoire) pour les deux datasets (sinon, il ne seront pas disjoints)  

**label_mode** détermine la forme des labels :
- **categorical** pour le multiclasses (utiliser une *loss* de type *categorical_crossentropy*)
- **int** pour des étiquettes au format entier (utiliser une *loss* de type *sparse_categorical_crossentropy*)
- **binary** pour une classification binaire (utiliser une *loss* de type *binary_crossentropy*)


In [ ]:
train_data_dir = "../input/cat-and-dog/training_set/training_set"
image_size = (299, 299)

train_dataset = image_dataset_from_directory(
    train_data_dir,
    validation_split=0.2,
    seed=1,
    subset="training",
    label_mode="categorical",
    image_size=image_size
)

validation_dataset = image_dataset_from_directory(
    train_data_dir,
    validation_split=0.2,
    seed=1,
    subset="validation",
    label_mode="categorical",
    image_size=image_size
)

## Amélioration des performances (optionnel)

Pour des datasets de grande taille, il est difficile de les charger en mémoire sans risquer de saturation de la mémoire  
C'est pourquoi on charge généralement les datasets par *batch* depuis le disque pour les traiter (avec des générateurs comme **ImageDataGenerator**). Néanmoins, la performance peut en souffrir, notamment avec l'utilisation de GPU, l'apprentissage étant principalement ralenti par les temps d'accès au disque.  
Avec **image_dataset_from_directory**, on peut régler la *prélecture* (*prefetch*) de manière à optimiser le séquençage des opérations entre lecture des données et apprentissage  
Le paramètre **AUTOTUNE** permet de régler dynamiquement la taille du buffer  
<img src="https://www.tensorflow.org/guide/images/data_performance/naive.svg">  

https://www.tensorflow.org/guide/data_performance   
http://restanalytics.com/2020-08-17-Transfer_Learning_and_Fine_tuning_withTensorFlow__Pneumonia_Classification_on_X_rays4/

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

## Modèle et entraînement

On crée une couche d'augmentation (attention : il n'est pas forcément souhaitable de réaliser trop de transformations, au risque de compromettre la convergence de l'entraînement) :

In [ ]:
data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1)
    RandomZoom((-0.1,0.1)),
    RandomContrast(0.05),  
    RandomTranslation(0.1,0.1)
])

On intègre la couche d'augmentation dans le modèle   
On ajoute également une couche **Rescaling**, puisque le dataset est généré "à la volée" (pour rappel, les algorithmes de descente du gradient convergent mieux pour des valeurs entre 0 et 1) :

In [ ]:
# Modèle CNN 
model = Sequential()
model.add(InputLayer(input_shape=(299, 299, 3)))
model.add(data_augmentation)
model.add(Rescaling(scale=1./255))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(20, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(2, activation='softmax', kernel_initializer=tf.keras.initializers.Constant(0.01)))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-3), metrics=['accuracy'])

On lance l'entraînement   
**Remarque :** l'augmentation **ne génère pas un dataset plus important en taille** ; à chaque *epoch*, on utilise un "nouveau" dataset, avec des images (légèrement) transformées

In [ ]:
history = model.fit(
    train_dataset, 
    validation_data=validation_dataset, 
    epochs=50,
    verbose=1)

In [ ]:
plot_scores(history)

Avec un modèle de type VGG16 :

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(299, 299, 3)))
model.add(data_augmentation)
model.add(Rescaling(scale=1./255))
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(2, activation='softmax'), kernel_initializer=tf.keras.initializers.Constant(0.01)))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-3), metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_dataset, 
    validation_data=validation_dataset, 
    epochs=50,
    verbose=1)

## Transfer learning

On va utiliser un modèle Xception   
https://towardsdatascience.com/review-xception-with-depthwise-separable-convolution-better-than-inception-v3-image-dc967dd42568

In [ ]:
from tensorflow.keras.applications import Xception
xception = Xception(weights='imagenet', include_top=False, input_shape=(299,299,3))
xception.trainable = False

In [ ]:
model = Sequential()
model.add(data_augmentation)
model.add(Rescaling(scale=1./255))
model.add(xception)
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_dataset, 
    validation_data=validation_dataset, 
    epochs=10,
    verbose=1)

In [ ]:
plot_scores(history)